# Analyze Angular Components and Generate a New Component

In this notebook, we will use the "Titan Text G1 - Premier" foundational model and the "Titan Embeddings G1 - Text" embeddings model to analyze the code structure of Angular components and generate a new component based on the analysis.

## Task 1: Environment Setup

First, we need to set up the environment by importing the necessary libraries and creating a service client.

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Task 2: Load and Analyze Angular Components

Next, we will load the Angular components from the specified directories and use the embeddings model to analyze their structure.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os

def load_all_files_recursive(directory):
    documents = []
    subfolders = set()  # Set to store unique subfolder names
    for root, dirs, files in os.walk(directory):
        # Skip hidden subfolders
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        for file in files:
            filepath = os.path.join(root, file)
            try:
                loader = TextLoader(filepath, encoding="utf-8")
                loaded_docs = loader.load()
                # Filter out empty documents
                non_empty_docs = [doc for doc in loaded_docs if doc.page_content.strip()]
                documents.extend(non_empty_docs)
                print(f"Parsed file {filepath}")
            except UnicodeDecodeError:
                print(f"Skipping binary file or file with unknown encoding: {filepath}")
            except Exception as e:
                print(f"Error loading file {filepath}: {e}")

        # Add subfolder names to the set
        for subfolder in dirs:
            subfolders.add(os.path.relpath(os.path.join(root, subfolder), directory))
    return documents, ", ".join(sorted(subfolders))

# Load ALL files recursively
documents, subfolders = load_all_files_recursive("../src/app")
print(f"Loaded {len(documents)} documents.")

# Create embeddings
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Create vector store
vectorstore_faiss = FAISS.from_documents(documents, embedding=br_embeddings)
print("Vector store created.")

# List all subfolders
print(f"\nSubfolders: {subfolders}")

# List all documents loaded to the vector store
print("\n--- Documents in Vector Store ---")
for doc in documents:
    print(f"Document Source: {doc.metadata.get('source', 'Unknown')}")
print("--- End of Documents in Vector Store ---")

## Task 3: Generate a New Component

Now, we will use the foundational model to generate a new Angular component based on the analysis stored in the vector database.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock

# Initialize chat model
chat_model = ChatBedrock(model_id="amazon.titan-text-premier-v1:0", client=bedrock_client)

# Define system prompt
system_prompt = (
    "You are an assistant for generating Angular components. "
    "Generate the component based on the human prompts. "
    "\n\n"
    "{context}"
)

# Define human prompt
human_prompt = (
    "Generate a component called with the name lava_flow. "
    "Use every single file within the " + subfolders + " and no other files "
    "from the context as a boilerplate for generating the component code. Ensure that "
    "the generated component has every single file that the boilerplate "
    "components have. Only output the filenames and their contents. "
    "Do not output anything else."
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create retrieval chain
retriever = vectorstore_faiss.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": human_prompt})

# Display the generated code in the specified format
generated_code = response['answer']

# Final generated code
print(generated_code)

## Conclusion

In this notebook, we successfully analyzed the structure of existing Angular components using the "Titan Embeddings G1 - Text" model and stored the analysis in a vector database. We then utilized the "Titan Text G1 - Premier" foundational model to generate a new Angular component based on the analyzed data. This approach demonstrates the potential of leveraging advanced language models to assist in software development tasks, such as code generation and component creation.